## 2.Data access

**Requirement:** First run #1

**Note**

On levante, one would not want to open data with xarray through xpublish because we can just do the same *open* command *without* xpublish. Cdo would be an exception (see cdo).

However, to run this notebook at one place, we use the node where you work on as the host. You can get the host url with the hostname of the respective levante node and the port that you used for the app. Accessing data through *eerie.cloud* works the same, just replace the `hosturl` with 'https://eerie.cloud.dkrz.de'.

In [1]:
port=9000
hostname=!echo $HOSTNAME
hosturl="https://"+hostname[0]+":"+str(port)
print(hosturl)

https://l40348.lvt.dkrz.de:9000


We have to tell the python programs to do not verify ssl certificates for our purposes:

In [2]:
storage_options=dict(verify_ssl=False)

### Xarray

Our example dataset is available via its name and the following `zarr_url`:

In [3]:
dsname="example"
zarr_url='/'.join([hosturl,"datasets",dsname,"zarr"])
print(zarr_url)

https://l40348.lvt.dkrz.de:9000/datasets/example/zarr


In [4]:
import xarray as xr
ds=xr.open_zarr(
    zarr_url,
    consolidated=True,
    storage_options=storage_options
)

### Intake

**All** datasets available through the app are collected in an intake catalog:

In [5]:
intake_url='/'.join([hosturl,"intake.yaml"])
print(intake_url)

https://l40348.lvt.dkrz.de:9000/intake.yaml


In [6]:
import intake
cat=intake.open_catalog(
    intake_url,
    storage_options=storage_options
)
list(cat)

['example']

In [7]:
cat[dsname](method="zarr",storage_options=storage_options).to_dask()

/work/bm0021/conda-envs/cloudify/lib/python3.11/site-packages/intake_xarray/base.py:21: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  'dims': dict(self._ds.dims),


<xarray.Dataset> Size: 314MB
Dimensions:    (lat: 192, time: 1032, bnds: 2, lon: 384)
Coordinates:
    height     float64 8B ...
  * lat        (lat) float64 2kB -89.28 -88.36 -87.42 ... 87.42 88.36 89.28
  * lon        (lon) float64 3kB 0.0 0.9375 1.875 2.812 ... 357.2 358.1 359.1
  * time       (time) datetime64[ns] 8kB 2015-01-16T12:00:00 ... 2100-12-16T1...
Dimensions without coordinates: bnds
Data variables:
    lat_bnds   (time, lat, bnds) float64 3MB dask.array<chunksize=(60, 192, 2), meta=np.ndarray>
    lon_bnds   (time, lon, bnds) float64 6MB dask.array<chunksize=(60, 384, 2), meta=np.ndarray>
    tas        (time, lat, lon) float32 304MB dask.array<chunksize=(1, 192, 384), meta=np.ndarray>
    time_bnds  (time, bnds) datetime64[ns] 17kB dask.array<chunksize=(1, 2), meta=np.ndarray>
Attributes: (12/47)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            ScenarioMIP AerChemMIP
    branch_method:          standard
    branch_time_in_child:   60265.0
    branch_time_in_parent:  60265.0
    contact:                cmip6-mpi-esm@dkrz.de
    ...                     ...
    title:                  MPI-ESM1-2-HR output prepared for CMIP6
    variable_id:            tas
    variant_label:          r1i1p1f1
    license:                CMIP6 model data produced by DKRZ is licensed und...
    cmor_version:           3.4.0
    tracking_id:            hdl:21.14100/d6f0f6b2-b4c2-460b-bc5e-f1c72f932399

### Stac

For each dataset, a stac item is generated with enriched metadata. The URL for this API is similar to the *zarr*-URL:

In [8]:
stac_url=zarr_url.replace('/zarr','/stac')

In [9]:
import pystac
import fsspec
import json
stacitem=pystac.item.Item.from_dict(
    json.load(fsspec.open(stac_url,**storage_options).open())
)
stacitem

<Item id=example>

In [10]:
stacitem.assets

{'data': <Asset href=https://eerie.cloud.dkrz.de/datasets/example/zarr>,
 'xarray_view': <Asset href=https://eerie.cloud.dkrz.de/datasets/example/>}

The stac API is right now hard-coded for 'eerie.cloud'. In theory, we could get to the data with xarray and the *href* asset.

### cdo

We are developing a cdo version that can read cloudified data with netcdf-zarr via http. The built netcdf is a testversion that prints some profiling warnings and hdf errors that often can be ignored. In 01/2025, it lacks port parsing which is why we cannot use it on internal apps on levante. Until this is solved, we can see how it works with the eerie.cloud:

In [3]:
cdo="/work/bm0021/cdo_incl_cmor/cdo-test_cmortest_gcc/bin/cdo"
hosturl="https://eerie.cloud.dkrz.de"
dsname="era5-dkrz.surface_analysis_hourly"
#
zarr_url='/'.join([hosturl,"datasets",dsname,"zarr"])
zarr_prefix="\#mode\=zarr,s3,consolidated"
infile=zarr_url+zarr_prefix
#
!{cdo} sinfo {infile}

   File format : NCZarr filter
    -1 : Institut Source   T Steptype Levels Num    Points Num Dtype : Parameter ID
     1 : unknown  ECMWF    v instant       1   1    542080   1  F64f : -1            
     2 : unknown  ECMWF    v instant       1   1    542080   1  F64f : -2            
     3 : unknown  ECMWF    v instant       1   1    542080   1  F64f : -3            
     4 : unknown  ECMWF    v instant       1   1    542080   1  F64f : -4            
     5 : unknown  ECMWF    v instant       1   1    542080   1  F64f : -5            
     6 : unknown  ECMWF    v instant       1   1    542080   1  F64f : -6            
     7 : unknown  ECMWF    v instant       1   1    542080   1  F64f : -7            
     8 : unknown  ECMWF    v instant       1   1    542080   1  F64f : -8            
     9 : unknown  ECMWF    v instant       1   1    542080   1  F64f : -9            
    10 : unknown  ECMWF    v instant       1   1    542080   1  F64f : -10           
    11 : unknown  ECMWF  

When working with cdo on cloudified data, we have to select data with the **select** operator:

In [6]:
!{cdo} info -select,name=2t,timestep=1 {infile}

cdo(1) select: Process started
    -1 :       Date     Time   Level Gridsize    Miss :     Minimum        Mean     Maximum : Parameter ID
     1 : 1940-01-01 00:00:00       0   542080       0 :      226.14      284.22      310.38 : -6            
cdo    info: Processed 542080 values from 1 variable over 1 timestep [7.16s 3675MB]
profiling:/home/k/k202186/repos:Cannot create directory
profiling:/home/k/k202186/repos/netcdf-c/build/plugins/CMakeFiles/h5deflate.dir/H5Zdeflate.c.gcda:Skip
profiling:/home/k/k202186/repos:Cannot create directory
profiling:/home/k/k202186/repos/netcdf-c/build/plugins/CMakeFiles/h5shuffle.dir/H5Zshuffle.c.gcda:Skip
profiling:/home/k/k202186/repos:Cannot create directory
profiling:/home/k/k202186/repos/netcdf-c/build/plugins/CMakeFiles/h5fletcher32.dir/H5checksum.c.gcda:Skip
profiling:/home/k/k202186/repos:Cannot create directory
profiling:/home/k/k202186/repos/netcdf-c/build/plugins/CMakeFiles/h5fletcher32.dir/H5Zfletcher32.c.gcda:Skip
profiling:/home/k/k20218